In [1]:
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
!pip install sentencepiece
import sentencepiece as spm
import torch
from torch.utils.data import Dataset,DataLoader

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.0MB 3.4MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
class Vocabulary:
  def __init__(self, path):
    self.paper = open(path, 'r').readlines()
    self.word2idx = {}
    self.word2count = {}
    self.idx2word = {'PAD_token': '<pad>', 'SOS_token': '<s>', 'EOS_token': '</s>'}
    self.num_words = 3
    self.longest_sentence = 0

  def postag(self, file):
    tag = [pos_tag(word_tokenize(f)) for f in file]
    tag[0].append(['[TITLE]'])
    tag[1].append(['[SEP]'])
    tags = []
    for sent in file:
      for word in sent:
        tags.append('/'.join(word))

    return tags

  def add_word(self, word):
    if word not in self.word2idx:
      self.word2idx[word] = self.num_words
      self.idx2word[self.num_words] = word
      self.num_words += 1

  def to_word(self, idx):
    return self.idx2word[idx]
  
  def to_idx(self, word):
    return self.word2idx[word]

  def __call__(self):
    for sent in self.postag(self.paper):
      for word in sent:
        self.add_word(word)

a = Vocabulary(path = 'drive/My Drive/Paper_Title_Extractor/data/train.dat')
a()

In [29]:
class PaperDataset(Dataset):
  def __init__(self, path, title_size, contents_size):
    self.paper = open(path, 'r').readlines()
    self.title_size = title_size
    self.contents_size = contents_size
    self.bos_id = '<s>'
    self.eos_id = '<s/>'
    self.pad_id = '<pad>'

  def __getitem__(self, index):
    line = self.paper[index]
    title, contents = line.split('\t')
    
    tag_title = self.postag(title)
    tag_contents = self.postag(contents)
    idx_title = self.idx(self, tag_title, self.title_size)
    idx_contents = self.idx(self, tag_contents, self.contents_size)
    return idx_title, idx_contents

  def __len__(self):
    return len(self.paper)

  def postag(self, file):
    tag = [pos_tag(word_tokenize(word)) for word in file]
    return '/'.join(tag).item()

  def idx(self, file, type):
    for word in file:
      idx = Vocabulary.to_idx(word)
      idx_pad = self.padding(self, idx, type)
    return idx_pad

  def padding(self, size, type):
    if len(size) >= type:
      size = size[:type]
    else:
      remainder_size = type - len(size)
      pad = [self.pad_id for none in range(remainder_size)]
      size = size + pad
    return size

dataset = PaperDataset(path = 'drive/My Drive/Paper_Title_Extractor/data/train.dat',
                         title_size = 8,
                         contents_size = 200)



TypeError: ignored

In [0]:
def data_loader():
  dataset = PaperDataset(path = 'drive/My Drive/Paper_Title_Extractor/data/train.dat',
                         title_size = 8,
                         contents_size = 200)
  
  data_load = DataLoader(dataset = dataset, batch_size = 100, shuffle = True)
  
  return data_load

a = data_loader()
a()